# Analyzing Residential Properties in Ames, Iowa

## Imports and Load Data

In [1]:
#Imports
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_log_error
from sklearn.pipeline import make_pipeline
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import tensorflow.keras.backend as K
from keras.layers import Dropout
from keras.layers import Dense
from keras.optimizers import Adam
from keras import regularizers
from xgboost import XGBRegressor
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Import data
train = 'https://raw.githubusercontent.com/ss16318/AML_Group_3/main/train.csv'
df_train_raw = pd.read_csv(train)

test_x = 'https://raw.githubusercontent.com/ss16318/AML_Group_3/main/test.csv'
df_test_x_raw = pd.read_csv(test_x)

test_y = 'https://raw.githubusercontent.com/ss16318/AML_Group_3/main/sample_submission.csv'
df_test_y_raw= pd.read_csv(test_y)

In [ ]:
# The original dataset on the Kaggle is divided into train and test set, we combine two dataset and split afterward in our way.
# Merge train dataset and test dataset
df_test_raw = pd.merge(df_test_x_raw, df_test_y_raw, on='Id')
df_raw = pd.concat([df_train_raw,df_test_raw])

## Clean Data

In [ ]:
# From the data description, when these values are nan, the property does not contain installment. 
# So we will change these features from nan to None
na_none = ['Alley','BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 
     'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [ ]:
# Check percentage of missing data for each feature in train dataset
na_col = [i for i in df_raw.columns if df_raw[i].isnull().sum() > 0 and i not in na_none]
print((df_raw[na_col].isna().sum()/len(df_raw.Id)).sort_values())

In [ ]:
def clean_df(df):
    
    for col in df.columns:
        
        # Replace the missing value to None if feature is in the na_none list
        if col in na_none:
            df[col] =  df[col].fillna('None')
        
        # Drop features that contain more than 20% missing values
        elif df[col].isnull().sum() / df.shape[0] > 0.1:
            df = df.drop(col, axis=1)
            
    return df

In [ ]:
df_cleaned = clean_df(df_raw) # drop columns and replace nan with none

df_cleaned = df_cleaned.dropna() #drop any row with missing data

In [ ]:
#Drop ID column
df_cleaned = df_cleaned.drop('Id', axis=1)

# split train dataset
df_x = df_cleaned.drop(columns='SalePrice')
df_y = df_cleaned['SalePrice']

df_x.shape

In [ ]:
df_cleaned

## EDA

In [ ]:
df_x.nunique(axis=0)

In [ ]:
df_x.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

### Univariate Analysis

In [ ]:
categorical = [
    'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
    'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
    'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
    'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'PavedDrive', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType',
    'SaleCondition'
]

numerical = [
    'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
    'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
    'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
    'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold',
]

plt.rcParams["figure.figsize"] = [8.00, 3.50]
plt.rcParams["figure.autolayout"] = True
sns.set(font_scale=3)

#### Categorical

In [ ]:
fig, ax = plt.subplots(8, 6, figsize=(50, 50))
for variable, subplot in zip(categorical, ax.flatten()):
    sns.barplot(x = variable, y = "SalePrice", data=df_cleaned, palette='rainbow', ax=subplot).set(title=f"{variable} Bar Chart")
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.show()

In [ ]:
specific_categorical = [
    'Neighborhood', 'CentralAir', 'Heating', 'HeatingQC', 'Electrical', 'MSZoning', 'LotConfig', 'LandSlope', 'RoofStyle'
]

fig, ax = plt.subplots(3, 3, figsize=(30, 30))
for variable, subplot in zip(specific_categorical, ax.flatten()):
    sns.barplot(x = variable, y = "SalePrice", data=df_cleaned, palette='rainbow', ax=subplot).set(title=f"{variable} Bar Chart")
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.show()

#### Numerical

In [ ]:
fig, ax = plt.subplots(6, 6, figsize=(100, 100))
for variable, subplot in zip(numerical, ax.flatten()):
    sns.histplot(x = variable, data=df_x, ax=subplot).set(title=f"{variable} Distribution")
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.show()

In [ ]:
specific_numerical = [
    'LotArea', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'MoSold', 'YearBuilt'
]

fig, ax = plt.subplots(3, 3, figsize=(30, 30))
for variable, subplot in zip(specific_numerical, ax.flatten()):
    sns.histplot(x = variable, data=df_x, ax=subplot).set(title=f"{variable} Distribution")
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.show()

### Bivariate Analysis

In [ ]:
fig, ax = plt.subplots(6, 6, figsize=(50, 50))
for variable, subplot in zip(numerical, ax.flatten()):
    sns.scatterplot(x=variable, y="SalePrice", data=df_cleaned, ax=subplot).set(title=f"{variable} vs Sale Price")
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.show()

In [ ]:
specific_bivariate_numerical = [
    'MoSold', 'YrSold', 'BedroomAbvGr', 'OverallQual', 'YearBuilt', 'GarageArea', 'GrLivArea', 'TotalBsmtSF'
]

fig, ax = plt.subplots(3, 3, figsize=(30, 30))
for variable, subplot in zip(specific_bivariate_numerical, ax.flatten()):
    sns.scatterplot(x=variable, y="SalePrice", data=df_cleaned, ax=subplot).set(title=f"{variable} vs Sale Price")
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.show()

#### Y-test summary stats

In [ ]:
# Plot a small multiple of bar charts
fig, ax = plt.subplots(1, 6, figsize=(12, 4))
for i, var in enumerate(['Street', 'MSSubClass', 'HouseStyle', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd']):
    df_cleaned[var].value_counts().plot(kind='bar', ax=ax[i])
    ax[i].set_title(var)

plt.tight_layout()
plt.show()

In [ ]:
# assume 'SalePrice' is the target variable in your dataframe
y = df_cleaned['SalePrice']

# create a histogram of the target variable
plt.hist(y, bins=50)
plt.xlabel('Sale Price')
plt.ylabel('Frequency')
plt.title('Distribution of Sale Price')
plt.show()

In [ ]:
#Calculating y-test summary stats
mean = np.mean(df_y)
median = np.median(df_y)
std_dev = np.std(df_y)
skewness = df_y.skew()
kurtosis = df_y.kurtosis()

print("Mean: ", mean)
print("Median: ", median)
print("Standard Deviation: ", std_dev)
print("Skewness: ", skewness)
print("Kurtosis: ", kurtosis)

## Encode Data

In [ ]:
#List categorical features
categorical_feats = df_x.select_dtypes('object').columns
categorical_feats

In [ ]:
# Encode variables

# Target Encode features that have many groups / are not ordinal
target_cat_feats = ['GarageType', 'Functional', 'Electrical', 'SaleCondition', 'SaleType', 'Heating', 'BsmtFinType2', 'BsmtFinType1', 'Foundation', 'MSZoning', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType']
target_encoder = ce.TargetEncoder(cols=target_cat_feats)

target_encoder.fit(df_x, df_y)
df_x_te = target_encoder.transform(df_x)

# Ordinal Encode features with few groups / have a natural order
ordinal_cat_feats = ['FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'Alley', 'KitchenQual', 'PavedDrive', 'GarageCond', 'GarageQual', 'GarageFinish', 'CentralAir', 'HeatingQC', 'BsmtExposure', 'BsmtCond', 'BsmtQual', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond']
ordinal_encoder = ce.OrdinalEncoder(cols=ordinal_cat_feats)

ordinal_encoder.fit(df_x_te, df_y)
df_x_encoded = ordinal_encoder.transform(df_x_te)

In [ ]:
df_x_encoded

In [ ]:
# We will probably to PCA, so looking at correlations is probably not too important

#plot correlation matrix
plt.figure(figsize=(100,100))
corr_matrix = df_x_encoded.corr()
sns.heatmap(corr_matrix, annot=True)
plt.show()

# Take absolute values of correlations and select upper triangle of correlation matrix
upper_matrix = corr_matrix.abs().where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.9
correlated_features = [column for column in upper_matrix.columns if any(upper_matrix[column] > 0.9)]
correlated_features

## Sort, Split and Scale Data

In [ ]:
# Sort Data Chronologically (preventing data leakage)
df_x_encoded = df_x_encoded.reset_index(drop=True)
df_y = df_y.reset_index(drop=True)

# Sort df_x_encoded by YearSold
df_x_sorted = df_x_encoded.sort_values(by='YrSold')

# Set the index of df_y to match the sorted index of df_x_sorted
df_y_sorted = df_y.loc[df_x_sorted.index]

In [ ]:
# Split Data into Dev and Test
X_dev, X_test, y_dev, y_test = train_test_split(df_x_sorted, df_y_sorted, test_size=0.2, shuffle=False)

# Split Dev into Train and Test
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.2, shuffle=False)

In [ ]:
#Scale Data
scale = StandardScaler()
X_train_scaled = scale.fit_transform(X_train)
X_val_scaled = scale.transform(X_val)
X_test_scaled = scale.fit_transform(X_test)

### Gaussian Mixture Modelling

In [ ]:
gmm = GaussianMixture(n_components=4, random_state=42)
gmm.fit(df_x_scaled)

# Predict the labels for the data
labels = gmm.predict(df_x_scaled)

# Create a list of y values for each cluster
y_cluster = [df_y[labels == i] for i in range(gmm.n_components)]

# Sort clusters in ascending order (by median)
sorted_y_cluster = sorted(y_cluster, key=lambda x: np.median(x))

# Boxplot of clusters
plt.boxplot(sorted_y_cluster)
plt.title("Boxplots of House Prices by Cluster")
plt.xlabel("Cluster")
plt.ylabel("House Price")
plt.show()

# Count number of points in each cluster
for i in range(0,len(sorted_y_cluster)):
    print("Cluster " + str(i+1) + ": " + str(len(sorted_y_cluster[i])) + " data points")  

## PCA

In [ ]:
from sklearn.decomposition import PCA

#Apply PCA to scaled data and compute the variance ratio:
pca = PCA()
pca.fit(X_train_scaled)
explained_variance_ratio = pca.explained_variance_ratio_

#Plot the explained variance ratio as a function of the number of principal components:
plt.plot(range(1, len(explained_variance_ratio) + 1), np.cumsum(explained_variance_ratio), label='Explained Variance')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.hlines(y=0.8, xmin=0, xmax=80, linestyles='dashed', colors='red', label='CEV Threshold')
plt.vlines(x=36, ymin=0, ymax=1, linestyles='dashed', colors='green', label='Component Cut-off')
plt.xlim((0,80))
plt.ylim((0,1.05))
plt.legend(loc='lower right')
plt.title('Explained Variance vs. Number of Principal Components')
plt.show()

In [ ]:
#Determine the number of PCA that explain a sufficient proportion of the variance
n_components = np.where(np.cumsum(explained_variance_ratio) >= 0.80)[0][0] + 1

In [ ]:
#Perform PCA with the optimal number of principal components:
pca_optimal = PCA(n_components=n_components)

X_train_pca = pca_optimal.fit_transform(X_train_scaled)
X_val_pca = pca_optimal.fit_transform(X_val_scaled)
X_test_pca = pca_optimal.fit_transform(X_test_scaled)

In [ ]:
# Create a DataFrame with principal components and columns named 'PC1', 'PC2', etc.
df_principal_components = pd.DataFrame(X_train_pca, columns=['PC' + str(i + 1) for i in range(n_components)])
df_principal_components

In [ ]:
#Important Features for PCA

#Get eigenvectors and eigenvalues
eigenvectors = pca.components_[:, :n_components]
eigenvalues = pca.explained_variance_[:n_components].reshape(-1, 1)

#Calculate feature importance
feature_imp = np.zeros((eigenvectors.shape[0]))
for i in range(0,len(eigenvalues)):
    feature_imp += np.abs(eigenvalues[i] * eigenvectors[:,i])

# Get the indices that would sort the feature_imp in ascending order
sort_ind = np.argsort(feature_imp)

# Select the last 10 indices (corresponding to the largest elements)
largest_feat_ind = sort_ind[-10:]
largest_feat_ind = largest_feat_ind[::-1]  # Reverse order to get largest to smallest

# Get the top 10 elements and their indices
most_imp = feature_imp[largest_feat_ind]

# Get a list of column names corresponding to the selected features
feat_names = list(df_x_encoded.columns[largest_feat_ind])

# Create the bar plot
fig = plt.figure(figsize=(8,5))
plt.bar(range(len(most_imp)), most_imp)
plt.xticks(range(len(feat_names)), feat_names)
plt.xlabel('Feature')
plt.title('Important Features for PCA')
plt.xticks(rotation=90)
plt.show()

## Linear Regression Models

### Linear regression on original data

In [ ]:
linear = LinearRegression()
linear.fit(X_train, y_train)

print("RMSLE: ", np.sqrt(mean_squared_log_error(y_test, linear.predict(X_test))))

In [ ]:
# plot feature importance
fig = plt.figure(figsize=(16,10))
linear_coefs = linear.coef_
names = df_x.columns.to_list()
ax = sns.barplot(x=names, y=linear_coefs)
plt.xticks(rotation=90)
plt.xlabel("Features")
plt.ylabel("Coefficient")
plt.show()

### LASSO regression on original data

In [ ]:
lasso = Lasso(random_state=0)
lasso_alpha = {'alpha': np.logspace(-5, 2, 20)}
grid_search_lasso = GridSearchCV(lasso, lasso_alpha)
grid_search_lasso.fit(X_val, y_val)

print("Best hyperparameters: ", grid_search_lasso.best_params_)

best_lasso = Lasso(alpha=grid_search_lasso.best_params_['alpha'], random_state=0)
best_lasso.fit(X_train, y_train)

print("RMSLE: ", np.sqrt(mean_squared_log_error(y_test, best_lasso.predict(X_test))))

In [ ]:
# plot feature importance
fig = plt.figure(figsize= (16,10))
lasso_coefs = best_lasso.coef_  
ax = sns.barplot(x=names, y=lasso_coefs)
plt.xticks(rotation=90)
plt.xlabel("Features")
plt.ylabel("Coefficient")
plt.show()

### Linear regression using Principal Components

In [ ]:
linear_pca = LinearRegression()
linear_pca.fit(X_train_pca, y_train)

print("RMSLE: ", np.sqrt(mean_squared_log_error(y_test, linear_pca.predict(X_test_pca))))

In [ ]:
# plot feature importance
fig = plt.figure(figsize= (16,8))
linear_coefs_pca = linear_pca.coef_
names = df_principal_components.columns.to_list()
ax = sns.barplot(x=names, y=linear_coefs_pca)
plt.xticks(rotation=90)
plt.xlabel("Features")
plt.ylabel("Coefficient")
plt.show()

### LASSO regression using Principal Components

In [ ]:
grid_search_lasso_pca = GridSearchCV(lasso, lasso_alpha)
grid_search_lasso_pca.fit(X_val_pca, y_val)

print("Best hyperparameters: ", grid_search_lasso_pca.best_params_)

best_lasso_pca = Lasso(alpha=grid_search_lasso_pca.best_params_['alpha'], random_state=0)
best_lasso_pca.fit(X_train_pca, y_train)

print("RMSLE: ", np.sqrt(mean_squared_log_error(y_test, best_lasso_pca.predict(X_test_pca))))

In [ ]:
# plot feature importance
fig = plt.figure(figsize= (16,8))
lasso_coefs_pca = best_lasso_pca.coef_
names = df_principal_components.columns.to_list()
ax = sns.barplot(x=names, y=lasso_coefs_pca)
plt.xticks(rotation=90)
plt.xlabel("Features")
plt.ylabel("Coefficient")
plt.show()

## Tree-Based Models

### Random Forest on original and PCA data

In [ ]:
n_estimators = [100, 200, 300, 400, 500]
max_depth = [3, 4, 5, 6, 7]
max_features = ["sqrt", "log2"]

param_combos = []
oob_scores = []
oob_scores_pca = []


for nm in n_estimators:
    for md in max_depth:
        for mf in max_features:
#             print(nm,md,mf)
            param_combos.append(f"{nm},{md},{mf}")
            
            # Build and train random forest model using base data
            rfr = RandomForestRegressor(n_estimators=nm,
                                       max_depth=md,
                                       max_features=mf,
                                       oob_score=True,
                                       warm_start=True)
            rfr.fit(X_train_scaled, y_train)
            
            # Using out-of-bag (OOB) error for finding the optimal hyperparameters
            oob_scores.append(rfr.oob_score_)
            
            del rfr
            
            # Build and train random forest model using PCA data
            rfr_pca = RandomForestRegressor(n_estimators=nm,
                                       max_depth=md,
                                       max_features=mf,
                                       oob_score=True,
                                       warm_start=True)
            rfr_pca.fit(X_train_pca, y_train)
            
            # Using out-of-bag (OOB) error for finding the optimal hyperparameters
            oob_scores_pca.append(rfr_pca.oob_score_)
            
            del rfr_pca
            
# Obtain index for maximum OOB-score
best_index = oob_scores.index(max(oob_scores))
best_index_pca = oob_scores_pca.index(max(oob_scores_pca))

# Obtain the best hyperparameters associated with that index
rfr_best_nm, rfr_best_md, rfr_best_mf = param_combos[best_index].split(',')
rfr_best_nm_pca, rfr_best_md_pca, rfr_best_mf_pca = param_combos[best_index_pca].split(',')

print(param_combos[best_index].split(','))
print(param_combos[best_index_pca].split(','))

#### Training Models with Best Hyperparameters

In [ ]:
'''
Best hyperparameters:
['300', '7', 'sqrt']
'''

best_rfr = RandomForestRegressor(n_estimators=int(rfr_best_nm),
                                max_depth=int(rfr_best_md),
                                max_features=rfr_best_mf)
best_rfr.fit(X_train_scaled, y_train)
rfr_y_pred = best_rfr.predict(X_test_scaled)
print('Root Mean Squared Logarithmic Error (RMSLE):', mean_squared_log_error(y_test, rfr_y_pred, squared=False))

In [ ]:
'''
Best hyperparameters:
['300', '3', '0.01']
'''

best_rfr_pca = RandomForestRegressor(n_estimators=int(rfr_best_nm_pca),
                                max_depth=int(rfr_best_md_pca),
                                max_features=rfr_best_mf_pca)
best_rfr_pca.fit(X_train_pca, y_train)
rfr_y_pred_pca = best_rfr_pca.predict(X_test_pca)
print('Root Mean Squared Logarithmic Error (RMSLE):', mean_squared_log_error(y_test, rfr_y_pred_pca, squared=False))

#### Graph Feature Importances

In [ ]:
# Organize and sort feature importances in descending order
rfr_tuned_importance = best_rfr.feature_importances_
feature_importance_dict = dict(zip(df_x.columns.to_list(), rfr_tuned_importance))
sorted_dict = dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True))

# Plot the feature importance graph
fig = plt.figure(figsize=(16,10))
plt.bar(sorted_dict.keys(),sorted_dict.values())
plt.xlabel("Features",labelpad = 20)
plt.ylabel("Feature Importance")
plt.title('Feature Importance for Random Forest Regression')
plt.xticks(rotation=90)
plt.figure(figsize=(50,50))
plt.show()

In [ ]:
# Organize and sort feature importances in descending order
rfr_tuned_importance = best_rfr.feature_importances_
feature_importance_dict = dict(zip(df_x.columns.to_list(), rfr_tuned_importance))
sorted_dict = dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:10])

# Plot the feature importance graph
fig = plt.figure(figsize=(8,5))
plt.bar(sorted_dict.keys(),sorted_dict.values())
plt.xlabel("Features",labelpad = 20)
plt.ylabel("Feature Importance")
plt.title('Top 10 Feature Importance for Random Forest Regression')
plt.xticks(rotation=90)

plt.show()

In [ ]:
pca_columns=['PC' + str(i + 1) for i in range(n_components)]
rfr_pca_tuned_importance = best_rfr_pca.feature_importances_

fig = plt.figure(figsize=(16,10))
plt.bar(pca_columns,rfr_pca_tuned_importance)
plt.xlabel("Features",labelpad = 20)
plt.ylabel("Feature Importance")
plt.title('Feature Importance for Random Forest Classifier with PCA')
plt.xticks(rotation=90)
plt.figure(figsize=(50,50))
plt.show()

### XGBoost on original and PCA data

In [ ]:
n_estimators = [100, 200, 300, 400, 500]
max_depth = [3, 4, 5, 6, 7]
learning_rate = [0.01, 0.1, 0.5]

param_combos = []
rmsle = []
rmsle_pca = []


for nm in n_estimators:
    for md in max_depth:
        for lr in learning_rate:
#             print(nm,md,lr)
            param_combos.append(f"{nm},{md},{lr}")
            
            # Build XGBoost model for base data
            xgbr = XGBRegressor(n_estimators=nm,
                               max_depth=md,
                               learning_rate=lr)
            xgbr.fit(X_train_scaled, y_train)
            
            # Using validation set and RMSLE for finding the optimal hyperparameters
            xgbr_y_pred = xgbr.predict(X_val_scaled)
            rmsle.append(mean_squared_log_error(y_val, xgbr_y_pred, squared=False))
            
            del xgbr

    
            # Build XGBoost model for PCA data
            xgbr_pca = XGBRegressor(n_estimators=nm,
                               max_depth=md,
                               learning_rate=lr)
            xgbr_pca.fit(X_train_pca, y_train)
            
            # Using validation set and RMSLE for finding the optimal hyperparameters
            xgbr_y_pred_pca = xgbr_pca.predict(X_val_pca)
            rmsle_pca.append(mean_squared_log_error(y_val, xgbr_y_pred_pca, squared=False))
            
            del xgbr_pca
            
            
# Obtain index for minimum RMSLE
best_index = rmsle.index(min(rmsle))
best_index_pca = rmsle_pca.index(min(rmsle_pca))

# Obtain the best hyperparameters associated with that index 
xgbr_best_nm, xgbr_best_md, xgbr_best_mf = param_combos[best_index].split(',')
xgbr_best_nm_pca, xgbr_best_md_pca, xgbr_best_mf_pca = param_combos[best_index_pca].split(',')

print(param_combos[best_index].split(','))
print(param_combos[best_index_pca].split(','))

#### Training Models with Best Hyperparameters

In [ ]:
'''
Best hyperparameters:
['200', '3', '0.1']
'''

best_xgbr = XGBRegressor(n_estimators=int(xgbr_best_nm),
                       max_depth=int(xgbr_best_md),
                       learning_rate=xgbr_best_mf)
best_xgbr.fit(X_train_scaled, y_train)
xgbr_best_y_pred = best_xgbr.predict(X_test_scaled)
print('Root Mean Squared Logarithmic Error (RMSLE):', mean_squared_log_error(y_test, xgbr_best_y_pred, squared=False))

In [ ]:
'''
Best hyperparameters:
['300', '3', '0.01']
'''

best_xgbr_pca = XGBRegressor(n_estimators=int(xgbr_best_nm_pca),
                       max_depth=int(xgbr_best_md_pca),
                       learning_rate=xgbr_best_mf_pca)
best_xgbr_pca.fit(X_train_pca, y_train)
xgbr_best_pca_y_pred = best_xgbr_pca.predict(X_test_pca)
print('Root Mean Squared Logarithmic Error (RMSLE):', mean_squared_log_error(y_test, xgbr_best_pca_y_pred, squared=False))

#### Graph Feature Importances

In [ ]:
xgb_tuned_importance = best_xgbr.feature_importances_

fig = plt.figure(figsize=(16,10))
plt.bar(df_x.columns.to_list(),xgb_tuned_importance)
plt.xlabel('Features',labelpad = 20)
plt.ylabel("Feature Importance")
plt.title('Feature Importance for XGBoost')
plt.xticks(rotation=90)
plt.figure(figsize=(50,50))
plt.show()

In [ ]:
xgb_pca_tuned_importance =best_xgbr_pca.feature_importances_

fig = plt.figure(figsize=(16,10))
plt.bar(pca_columns,xgb_pca_tuned_importance)
plt.xlabel("Features",labelpad = 20)
plt.ylabel("Feature Importance")
plt.title('Feature Importance for XBoost with PCA')
plt.xticks(rotation=90)
plt.figure(figsize=(50,50))
plt.show()

## Neural Network Approach

In [ ]:
# Define Root Mean Squared Logarithmic Error in TF
def rmsle(y_true, y_pred):
    return K.sqrt(K.mean(K.square(K.log(y_pred + 1) - K.log(y_true + 1))))

In [ ]:
def build_NN(x_train, reg_param, drop_out, lr, pca):
    
    model = Sequential()
    
    if pca == 0: 
        # The Input Layer :
        model.add(Dense(78, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu', kernel_regularizer=regularizers.l1(reg_param)))

        # The Hidden Layer:
        model.add(Dense(32, kernel_initializer='normal',activation='relu', kernel_regularizer=regularizers.l1(reg_param)))
        model.add(Dropout(drop_out))
        
    if pca == 1:
        model.add(Dense(36, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu', kernel_regularizer=regularizers.l1(reg_param)))

    # Hidden Layers
    model.add(Dense(16, kernel_initializer='normal',activation='relu', kernel_regularizer=regularizers.l1(reg_param)))
    model.add(Dropout(drop_out))
    
    model.add(Dense(8, kernel_initializer='normal',activation='relu', kernel_regularizer=regularizers.l1(reg_param)))
    model.add(Dropout(drop_out))
    
    model.add(Dense(4, kernel_initializer='normal',activation='relu', kernel_regularizer=regularizers.l1(reg_param)))

    # The Output Layer :
    model.add(Dense(1, kernel_initializer='normal',activation='linear'))
    
    #Define Learning Rate of Optimizer
    optimizer = Adam(learning_rate=lr)
    
    #Compile Model
    model.compile(optimizer=optimizer, loss=rmsle, metrics=[rmsle])
    
    return model

In [ ]:
# Show example structure
model_example = model = build_NN(X_train, 0, 0, 0, pca=0)
model_example.summary()

In [ ]:
# This code takes a long time to run and sometimes yields inconsistent results/errors
# In the next cell are the tuned parameters used for results shown in the report
# They are a combination of results from this grid search tuning approach and manual
# trial and error

# #Hyperparameters
# reg_param = [0, 0.01, 0.001, 0.0001]
# drop_outs = [0, 0.1, 0.2, 0.3]
# lrs = [0.1, 0.01, 0.001, 0.0001, 0.00001]
# epochs = [10, 20, 50, 100]

# best_rmsle = 100
# best_rmsle_pca = 100

# for rp in reg_param:
#     for drop_out in drop_outs:
#         for lr in lrs:
#             for epoch in epochs:

#                 #Model without PCA
                
#                 #Create Model
#                 model = build_NN(X_train, rp, drop_out, lr, 0)

#                 # Train model
#                 model.fit(X_train, y_train, epochs=epoch, verbose=0)

#                 # Evaluate model
#                 val_loss, val_rmsle = model.evaluate(X_val, y_val)

#                 if val_rmsle < best_rmsle:

#                     best_reg = rp
#                     best_do = drop_out
#                     best_lr = lr
#                     best_epoch = epoch

#                     best_rmsle = val_rmsle

#                 del model
#                 del val_rmsle

#                 #Model with PCA
                
#                 #Create Model
#                 model = build_NN(X_train_pca, rp, drop_out, lr, 1)

#                 # Train your model
#                 model.fit(X_train_pca, y_train, epochs=epoch, verbose=0)

#                 # Evaluate your model using RMSLE
#                 val_loss, val_rmsle = model.evaluate(X_val_pca, y_val)

#                 if val_rmsle < best_rmsle_pca:
                    
#                     best_reg_pca = rp
#                     best_do_pca = drop_out
#                     best_lr_pca = lr
#                     best_epoch = epoch

#                     best_rmsle_pca = val_rmsle

#                 del model
#                 del val_rmsle

In [ ]:
# Best Parameters for Hyperparameter Tuning (model without PCA)
reg_param = 0.0001
drop_out = 0.1
learning_rate = 0.01
epoch = 10

In [ ]:
model1 = build_NN(X_train, reg_param, drop_out, learning_rate, 0)

# Train your model and store the history object
history = model1.fit(X_train, y_train, epochs=epoch, verbose=0, validation_data=(X_val, y_val))

# Get the training and validation loss from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the training and validation loss as a function of epoch
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss Plots for Data Without PCA')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss, test_rmsle = model1.evaluate(X_test, y_test)

In [ ]:
# Best Parameters for Hyperparameter Tuning (model with PCA)
reg_param = 0.01
drop_out = 0.2
learning_rate = 0.01
epoch = 20

In [ ]:
model2 = build_NN(X_train_pca, reg_param, drop_out, learning_rate, 1)

# Train your model and store the history object
history = model2.fit(X_train_pca, y_train, epochs=epoch, verbose=0, validation_data=(X_val_pca, y_val))

# Get the training and validation loss from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the training and validation loss as a function of epoch
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss Plots for Data Without PCA')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss, test_rmsle = model2.evaluate(X_test_pca, y_test)

## Group-Specific 

In [ ]:
from sklearn.mixture import GaussianMixture

#Gaussian Mixture Modelling
gmm = GaussianMixture(n_components=4, random_state=42)
gmm.fit(X_train_scaled)

# Predict the labels for the data
labels = gmm.predict(X_train_scaled)

# Create a list of y values for each cluster
y_cluster = [y_train[labels == i] for i in range(gmm.n_components)]

# Sort clusters in ascending order (by median)
sorted_y_cluster = sorted(y_cluster, key=lambda x: np.median(x))

# Boxplot of clusters
plt.boxplot(sorted_y_cluster)
plt.title("Boxplots of House Prices by Cluster")
plt.xlabel("Cluster")
plt.ylabel("House Price")
plt.show()

# Count number of points in each cluster
for i in range(0,len(sorted_y_cluster)):
    print("Cluster " + str(i+1) + ": " + str(len(sorted_y_cluster[i])) + " data points")  

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
import matplotlib.pyplot as plt
import numpy as np

# Initialize a list to store the RMSLE for each cluster-specific model
rmsle_list = []

# Loop through each cluster
for i in range(gmm.n_components):
    # Get the indices of the samples belonging to the current cluster
    cluster_indices = np.where(labels == i)[0]

    # Split the data into training and testing sets for the current cluster
    X_cluster_train, X_cluster_test, y_cluster_train, y_cluster_test = train_test_split(
        X_train.iloc[cluster_indices], y_train.iloc[cluster_indices], test_size=0.2, random_state=42)

    # Train a linear regression model for the current cluster
    linear_cluster = LinearRegression()
    linear_cluster.fit(X_cluster_train, y_cluster_train)

    # Calculate the RMSLE for the current cluster-specific model
    y_pred = linear_cluster.predict(X_cluster_test)
    y_pred_clipped = np.clip(y_pred, 0, None)
    rmsle = np.sqrt(mean_squared_log_error(y_cluster_test, y_pred_clipped))

    rmsle_list.append(rmsle)
    print(f"Cluster {i+1} RMSLE: {rmsle}")

# Calculate the average RMSLE across all cluster-specific models
average_rmsle = np.mean(rmsle_list)
print(f"Average RMSLE for cluster-specific models: {average_rmsle}")

# Compare the average RMSLE with the baseline RMSLE
baseline_rmsle = np.sqrt(mean_squared_log_error(y_test, linear.predict(X_test)))
print(f"Baseline RMSLE: {baseline_rmsle}")

Based on the results you provided, the cluster-specific linear regression models have a higher average RMSLE (0.5308) compared to the baseline linear regression model (0.1915). This indicates that the group-specific regression for each cluster found in the GMM does not enhance the performance compared to the linear regression on the original data in this case.

One possible reason for this outcome is that the GMM clustering is not effectively capturing the underlying structure of the data in a way that benefits the prediction of house prices. Another reason could be the presence of a cluster with a particularly high RMSLE (Cluster 4) which is skewing the average RMSLE upwards. You could try experimenting with different clustering techniques or varying the number of clusters to see if you can obtain better results.

In [ ]:
# Initialize a list to store the RMSLE for each cluster-specific model
rmsle_list = []

# Loop through each cluster, excluding Cluster 4
for i in range(gmm.n_components - 1):  # Change the loop range to exclude the last cluster
    # Get the indices of the samples belonging to the current cluster
    cluster_indices = np.where(labels == i)[0]

    # Split the data into training and testing sets for the current cluster
    X_cluster_train, X_cluster_test, y_cluster_train, y_cluster_test = train_test_split(
        X_train.iloc[cluster_indices], y_train.iloc[cluster_indices], test_size=0.2, random_state=42)

    # Train a linear regression model for the current cluster
    linear_cluster = LinearRegression()
    linear_cluster.fit(X_cluster_train, y_cluster_train)

    # Calculate the RMSLE for the current cluster-specific model
    y_pred = linear_cluster.predict(X_cluster_test)
    y_pred_clipped = np.clip(y_pred, 0, None)
    rmsle = np.sqrt(mean_squared_log_error(y_cluster_test, y_pred_clipped))

    rmsle_list.append(rmsle)
    print(f"Cluster {i+1} RMSLE: {rmsle}")

# Calculate the average RMSLE across all cluster-specific models, excluding Cluster 4
average_rmsle_without_cluster4 = np.mean(rmsle_list)
print(f"Average RMSLE for cluster-specific models without Cluster 4: {average_rmsle_without_cluster4}")

# Compare the average RMSLE without Cluster 4 with the baseline RMSLE
print(f"Baseline RMSLE: {baseline_rmsle}")


Based on the updated results without Cluster 4, the average RMSLE for the cluster-specific linear regression models is 0.2205, which is still higher than the baseline RMSLE of 0.1915. This indicates that even without considering Cluster 4, the group-specific regression for each cluster found in the GMM does not enhance the performance compared to the linear regression on the original data.

This could be because the GMM clustering is still not effectively capturing the underlying structure of the data in a way that benefits the prediction of house prices. It may be worth exploring other clustering techniques, varying the number of clusters, or even trying different regression algorithms to see if you can obtain better results.

In [ ]:
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define clustering algorithms with their specific parameters
clustering_algorithms = {
    'KMeans': {'algorithm': KMeans, 'params': {'n_clusters': None, 'random_state': 42}},
    'AgglomerativeClustering': {'algorithm': AgglomerativeClustering, 'params': {'n_clusters': None}},
    'GaussianMixture': {'algorithm': GaussianMixture, 'params': {'n_components': None, 'random_state': 42}}
}

# Define regression algorithms
regression_algorithms = {
    'LinearRegression': LinearRegression,
    'LASSO': Lasso,
    'RandomForestRegressor': RandomForestRegressor,
    'XGBoost': XGBRegressor,
    'NeuralNetworkRegressor': MLPRegressor
}

# Define the range of the number of clusters to explore
n_clusters_range = range(2, 8)

# Initialize variables to store the best model information
best_rmsle = float('inf')
best_clustering = None
best_n_clusters = None
best_regression = None

# Loop through each clustering algorithm
for clustering_name, clustering_info in clustering_algorithms.items():
    # Loop through the range of the number of clusters
    for n_clusters in n_clusters_range:
        # Set the appropriate parameter for the number of clusters or components
        if clustering_name == 'GaussianMixture':
            clustering_info['params']['n_components'] = n_clusters
        else:
            clustering_info['params']['n_clusters'] = n_clusters

        # Initialize the clustering algorithm with the current number of clusters and specific parameters
        clustering = clustering_info['algorithm'](**clustering_info['params'])

        # Fit the clustering algorithm to the training data and predict the labels
        if clustering_name == 'GaussianMixture':
            clustering.fit(X_train_scaled)
            labels = clustering.predict(X_train_scaled)
        else:
            labels = clustering.fit_predict(X_train_scaled)
        
        # Calculate the silhouette score for the current clustering
        silhouette_avg = silhouette_score(X_train_scaled, labels)
        
        # Loop through each regression algorithm
        for regression_name, Regression in regression_algorithms.items():
            # Initialize a list to store the RMSLE for each cluster-specific model
            rmsle_list = []
            
            # Loop through each cluster
            for i in range(n_clusters):
                # Get the indices of the samples belonging to the current cluster
                cluster_indices = np.where(labels == i)[0]

                # Split the data into training and testing sets for the current cluster
                X_cluster_train, X_cluster_test, y_cluster_train, y_cluster_test = train_test_split(
                    X_train.iloc[cluster_indices], y_train.iloc[cluster_indices], test_size=0.2, random_state=42)

                # Train a regression model for the current cluster
                regression = Regression()
                regression.fit(X_cluster_train, y_cluster_train)

                # Calculate the RMSLE for the current cluster-specific model
                y_pred = regression.predict(X_cluster_test)
                y_pred_clipped = np.clip(y_pred, 0, None)
                rmsle = np.sqrt(mean_squared_log_error(y_cluster_test, y_pred_clipped))

                rmsle_list.append(rmsle)

            # Calculate the average RMSLE across all cluster-specific models
            average_rmsle = np.mean(rmsle_list)
            
            # Update the best model information if the current combination has a lower average RMSLE
            if average_rmsle < best_rmsle:
                best_rmsle = average_rmsle
                best_clustering = clustering_name
                best_n_clusters = n_clusters
                best_regression = regression_name

            print(f"{clustering_name} with {n_clusters} clusters and {regression_name}: Average RMSLE = {average_rmsle:.4f}, Silhouette Score = {silhouette_avg:.4f}")

# Print the best model information
print(f"\nBest combination: {best_clustering} with {best_n_clusters} clusters and {best_regression}")
print(f"Best average RMSLE: {best_rmsle:.4f}")
